## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [52]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess, source_localization

### Paths

In [53]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


In [54]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [55]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_dir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [56]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 37
Controls: 25
Total: 62


#### Look for subjects who do not have EO or EC cropped data

In [57]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

[]
[]


In [58]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             'lateraloccipital-lh', # Left Lateral-OCC CONTROL
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh', # Right Medial-OFC
             'lateraloccipital-rh', # Right Lateral-OCC CONTROL
]


### Compute STCs

In [59]:
nan_subjects=[]

methods = [
    # 'dSPM',
    'MNE',
    ]

for sub_id in sub_ids: 
    # Check if eyes open or eyes closed need to be computed
    EC_bool = False if sub_id in no_eyes_open else True
    EO_bool = False if sub_id in no_eyes_closed else True
    print(f"Subject: {sub_id} | EC: {EC_bool} | EO: {EO_bool}")
    
    for save_stc_mat in [
        False,
        # True,
        ]:    
        for method in methods:
            # Cannot compute without eyes open if using dSPM
            if not EO_bool and method=='dSPM':
                continue

            # else, compute
            print(f"Source Localization Method: {method}")                          
            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method})')
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 

            # Create paths
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = False # TODO: temporary for resting control

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = source_localization.to_source(sub_id,
                                                                              processed_data_path,
                                                                              zscored_epochs_save_path,
                                                                              EC_resting_save_path,
                                                                              EO_resting_save_path,
                                                                              roi_names,
                                                                              times_tup,
                                                                              method=method,
                                                                              return_zepochs=Epochs_bool,
                                                                              return_EC_resting = EC_bool,
                                                                              return_EO_resting = EO_bool,
                                                                              average_dipoles=average_dipoles,
                                                                              save_stc_mat=save_stc_mat,
                                                                              save_inv=save_inv,
                                                                              )
            utils.clear_display()

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 20485/20484 = 2.207463
    scale = 125765 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 3.5e-13 (2.2e-16 eps * 64 dim * 25  max singular value)
    Estimated rank (eeg): 60
    EEG: rank 60 computed from 64 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of w

### Assess results

In [60]:
nan_subjects

[]

In [61]:
label_ts_All_Conds

(None,
 array([[ 6.64263428e-12,  5.11119894e-12,  5.12270111e-12, ...,
          9.67194405e-12,  7.47677763e-12,  3.64382687e-12],
        [ 1.13726147e-11,  9.70478547e-12,  7.28833090e-12, ...,
          4.82271480e-12,  3.22019125e-12,  3.44218318e-12],
        [ 1.74680009e-12,  6.29592737e-13, -4.28604311e-13, ...,
          2.63713947e-12,  2.18956846e-12,  1.82310109e-12],
        ...,
        [-9.44623888e-13, -1.05302848e-12, -1.30318606e-12, ...,
         -1.64596775e-12, -1.01085612e-12, -4.22806505e-13],
        [ 3.86106735e-12,  3.32597388e-12,  3.49988034e-12, ...,
          6.79912830e-12,  4.97349158e-12,  1.31556269e-12],
        [ 1.13035654e-11,  7.86009427e-12,  6.60738394e-12, ...,
          6.27137591e-12,  8.92752297e-12,  1.02533786e-11]]),
 array([[ 4.40433638e-12,  5.46878617e-12,  2.95190819e-12, ...,
          1.06752586e-11,  1.10389081e-11,  9.01491001e-12],
        [ 1.41547636e-12,  3.12651975e-12,  3.62969247e-12, ...,
          1.29102518e-11,  1.28